In [1]:
import json 
import os
from gen_util import * 


In [2]:
class Config_Writer:
    def __init__(self,folder):
        self.folder = folder 
    def write(self,file_name,config_dict):
        path = os.path.join(self.folder,file_name)
        if not os.path.exists(self.folder):
            os.makedirs(self.folder)
        with open(os.path.join(self.folder,file_name),"w") as f:
            json.dump(config_dict,f)

In [3]:
path_v2 = "/home/renming.qi/models/pretrained/inception_resnet_v2/inception_resnet_v2_2016_08_30.ckpt"
path_v3 = "/home/renming.qi/models/pretrained/inception_v3/inception_v3.ckpt"
path_mobile = "/home/renming.qi/models/pretrained/mobilenet_v1/mobilenet_v1_1.0_224.ckpt"


In [4]:
import copy
def gen_validation_and_train_eval(eval_config,csv_path_middle,csv_path_final):
    eval_config['labels_offset']="0"
    eval_config_val_middle = copy.deepcopy(eval_config)
    eval_config_val_final = copy.deepcopy(eval_config)
    eval_config_train_middle = copy.deepcopy(eval_config)
    eval_config_train_final = copy.deepcopy(eval_config)
    eval_config_val_middle['dataset_split_name'] = "validation"
    eval_config_val_middle['csv_name'] = csv_path_middle + "_val.csv"
    eval_config_val_final['dataset_split_name'] = "validation"
    eval_config_val_final['csv_name'] = csv_path_final + "_val.csv"
    eval_config_train_middle['dataset_split_name']="train"
    eval_config_train_middle['csv_name'] = csv_path_middle + "_train.csv"
    eval_config_train_final['dataset_split_name']="train"
    eval_config_train_final['csv_name'] = csv_path_final + "_train.csv"
    return eval_config_val_middle,eval_config_train_middle,eval_config_val_final,eval_config_train_final
def gen_forzen_and_unfrozen_train(train_config):
    train_config["labels_offset"]="0"
    
    train_config_frozen = copy.deepcopy(train_config)
    train_config_frozen["max_number_of_steps"] = str(50)
    train_config_unfrozen = copy.deepcopy(train_config)
    train_config_unfrozen['checkpoint_path'] = train_config_frozen['train_dir']
    train_config_unfrozen["learning_rate"]="0.0001"
    train_config_unfrozen["max_number_of_steps"] = str(50)
    return train_config_frozen, train_config_unfrozen

In [5]:
DATASET_DIR="/home/renming.qi/flower_photos"

config_writer = Config_Writer("test_v2")
train_folder ="test_v2"
ck_path , train_p, eval_p,csv_path_middle = path_gen(path_v2,train_folder=train_folder, csv_name_prefix ="middle")
ck_path , train_p, eval_p,csv_path_final = path_gen(path_v2,train_folder=train_folder, csv_name_prefix = "final")

config_gen = ConfigFactory()
train_config = config_gen.gen_fn("train:inception_resnet_v2")
train_config = train_config(ck_path,DATASET_DIR,train_p)

config_gen = ConfigFactory()
eval_config = config_gen.gen_fn("eval:inception_resnet_v2")
eval_config = eval_config(train_p,DATASET_DIR,eval_p)

evm,etm,evf,etf = gen_validation_and_train_eval(eval_config,csv_path_middle,csv_path_final)
tfz, tuf = gen_forzen_and_unfrozen_train(train_config)
task_seq = [tfz,evm,etm,tuf,evf,etf]
task_describe = {"task_name":"test_mobile","task_seq":task_seq}

config_writer = Config_Writer("./test_v2")
config_writer.write("inception_resnet_v2.json",task_describe)

In [6]:
runner = SeqRunner(task_dict=task_describe)
runner.run()

_______________
{'max_number_of_steps': '50', 'trainable_scopes': 'InceptionResnetV2/Logits,InceptionResnetV2/AuxLogits', 'learning_rate_decay_type': 'fixed', 'optimizer': 'rmsprop', 'checkpoint_exclude_scopes': 'InceptionResnetV2/Logits,InceptionResnetV2/AuxLogits', 'dataset_split_name': 'train', 'save_summaries_secs': '60', 'learning_rate': '0.01', 'train_dir': './test_v2', 'batch_size': '32', 'method': 'train', 'labels_offset': '0', 'preprocessing_name': 'inception', 'dataset_dir': '/home/renming.qi/flower_photos', 'save_interval_secs': '600', 'weight_decay': '0.00004', 'model_name': 'inception_resnet_v2', 'dataset_name': 'rcmp'}
_______________
_______________
{'checkpoint_path': './test_v2', 'eval_dir': './test_v2', 'dataset_split_name': 'validation', 'csv_name': './test_v2/middle_val.csv', 'model_name': 'inception_resnet_v2', 'labels_offset': '0', 'dataset_dir': '/home/renming.qi/flower_photos', 'method': 'eval', 'dataset_name': 'rcmp'}
_______________
_______________
{'checkpoin

In [ ]:
DATASET_DIR="/home/renming.qi/flower_photos"

config_writer = Config_Writer("test_v3")
train_folder ="test_v3"
ck_path , train_p, eval_p,csv_path_middle = path_gen(path_v3,train_folder=train_folder, csv_name_prefix ="middle")
ck_path , train_p, eval_p,csv_path_final = path_gen(path_v3,train_folder=train_folder, csv_name_prefix = "final")

config_gen = ConfigFactory()
train_config = config_gen.gen_fn("train:inception_v3")
train_config = train_config(ck_path,DATASET_DIR,train_p)

config_gen = ConfigFactory()
eval_config = config_gen.gen_fn("eval:inception_v3")
eval_config = eval_config(train_p,DATASET_DIR,eval_p)

evm,etm,evf,etf = gen_validation_and_train_eval(eval_config,csv_path_middle,csv_path_final)
tfz, tuf = gen_forzen_and_unfrozen_train(train_config)
task_seq = [tfz,evm,etm,tuf,evf,etf]
task_describe = {"task_name":"test_mobile","task_seq":task_seq}


config_writer = Config_Writer("./test_v3")
config_writer.write("inception_v3.json",task_describe)
runner = SeqRunner(task_dict=task_describe)
runner.run()

In [ ]:
DATASET_DIR="/home/renming.qi/flower_photos"

config_writer = Config_Writer("test_mob_v1")
train_folder ="test_mob_v1"
ck_path , train_p, eval_p,csv_path_middle = path_gen(path_mobile,train_folder=train_folder, csv_name_prefix ="middle")
ck_path , train_p, eval_p,csv_path_final = path_gen(path_mobile,train_folder=train_folder, csv_name_prefix = "final")

config_gen = ConfigFactory()
train_config = config_gen.gen_fn("train:mobile_net_v1")
train_config = train_config(ck_path,DATASET_DIR,train_p)

config_gen = ConfigFactory()
eval_config = config_gen.gen_fn("eval:mobile_net_v1")
eval_config = eval_config(train_p,DATASET_DIR,eval_p)

evm,etm,evf,etf = gen_validation_and_train_eval(eval_config,csv_path_middle,csv_path_final)
tfz, tuf = gen_forzen_and_unfrozen_train(train_config)
task_seq = [tfz,evm,etm,tuf,evf,etf]
task_describe = {"task_name":"test_mobile","task_seq":task_seq}


config_writer = Config_Writer("./test_mobile")
config_writer.write("mobile.json",task_describe)
runner = SeqRunner(task_dict=task_describe)
runner.run()